# Context generation for Retrieval Augmented Generation (RAG)
RAG typically retrieves the related contexts when prompting.
However, that requires the loading of 2 models which requires ~48GB of RAM
To avoid CUDA out of memory error, this pipeline seperates the retrieval from the actual prompting.

As such the retrieval is only based on the original description of the crop from GSA and not the entire prompt.
This example calls the PrecomputedRAG function to find the top-k related texts saved in the FAISS vector store generated earlier.
In PrecomputedRAG, the query is turned into embeddings using Qwen3 Embedding 8B and matched based on cosine similarity to the most similar texts in the FAISS store.

In [ ]:

from pathlib import Path

import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

from generate_context import PrecomputedRAG

/opt/miniconda3/envs/llm/lib/python3.10/site-packages/transformers/utils/hub.py:105: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
index_path = '/home/automate_eurocrops/data/indices/agrovoc_agriprod_faiss.index'

data_path = '/home/automate_eurocrops/data/indices/agrovoc_agriprod_metadata.pkl'

data_root = '/home/automate_eurocrops/data/raw/'
BW_name = 'BW.csv'
BW = pd.read_csv(data_root+BW_name)

original_name = BW['original_name']
model_name = 'Qwen/Qwen3-Embedding-8B'



In [3]:
preRAG = PrecomputedRAG(data_path, index_path,model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loaded 31018 precomputed embeddings from metadata.
FAISS index: /home/eouser/tabVol/automate_eurocrops/data/indices/agrovoc_agriprod_faiss.index
Query encoder: Qwen/Qwen3-Embedding-8B


In [15]:
test_crop = original_name [4]
context_s = preRAG.get_context_string(test_crop)
print('Query crop: ', test_crop)
print(context_s)

Query crop:  Winterweichweizen
primitive wheat's german translation is alte Weizenart, primitive wheat's french translation is blé primitif, primitive wheat's includes is Dasypyrum villosum, primitive wheat's includes in german is Dasypyrum villosum, primitive wheat's includes in french is Dasypyrum villosum

primitive wheat's german translation is alte Weizenart, primitive wheat's french translation is blé primitif, primitive wheat's includes is Triticum compactum, primitive wheat's includes in german is Triticum compactum, primitive wheat's includes in french is Triticum compactum

Winter durum wheat's french name is Blé dur d'hiver, Winter durum wheat's german name is Winterhartweizen

Common winter wheat and spelt's french name is Blé tendre et épeautre d'hiver, Common winter wheat and spelt's german name is Winterweichweizen und Dinkel

Winter triticale's french name is Triticale d'hiver, Winter triticale's german name is Wintertriticale
